In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import warnings
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn import metrics
from imblearn.combine import SMOTEENN
import gc
%matplotlib inline

warnings.filterwarnings("ignore")



def feat_select(train, test):
    features = train.drop(['is_trade', 'time', 'context_timestamp','instance_id'], axis=1).columns.tolist()
    feature = []
    target = ['is_trade']

    return features, target


def xgbCV(train, test):
    features, target = feat_select(train, test)

    X = train[features]
    y = train[target]
    X_tes = test[features]
    y_tes = test[target]

    print('Training XGB model...')
    X_train_set = xgb.DMatrix(X, label=y, missing=np.nan)
    X_validate_set = xgb.DMatrix(X_tes, label=y_tes, missing=np.nan)
    watchlist = [(X_train_set, 'train'), (X_validate_set, 'eval')]
    params = {'max_depth': 7,
              'nthread': -1,
              'eta': 0.01,
              'eval_metric': 'auc',
              'objective': 'binary:logistic',
              'subsample': 0.85,
              'colsample_bytree': 0.85,
              'silent': 1,
              'seed': 0,
              'min_child_weight': 6,
              'gpu_id': 0,
              'tree_method': 'gpu_hist',
#               'scale_pos_weight':20
              }
    gbm = xgb.train(params, X_train_set, num_boost_round=3000, evals=watchlist, early_stopping_rounds=50)

    best_iter_num = gbm.best_iteration
    return best_iter_num


def sub(train, test, best_iter_num):
    features, target = feat_select(train, test)

    X = train[features]
    y = train[target]
    X_train_set = xgb.DMatrix(X, label=y, missing=np.nan)
    X_test_set = xgb.DMatrix(test[features], missing=np.nan)
    print('Training XGB model...')
    params = {'max_depth': 7,
              'nthread': -1,
              'eta': 0.01,
              'eval_metric': 'auc',
              'objective': 'binary:logistic',
              'subsample': 0.85,
              'colsample_bytree': 0.85,
              'silent': 1,
              'seed': 0,
              'min_child_weight': 6,
              'gpu_id': 0,
              'tree_method': 'gpu_hist',
#               'scale_pos_weight':20
              }
    gbm = xgb.train(params, X_train_set, num_boost_round=best_iter_num)
    gbm.save_model('001.model')

In [2]:
path = './data/'
data = pd.read_csv(path + 'all_final_data_11-07-18-51.csv')
data = data[data.is_trade.notnull()]
train_data, test_data = train_test_split(data, test_size=0.25)

In [3]:
data[data.is_trade==1].shape

(9499, 250)

In [4]:
data[data.is_trade==0].shape

(469516, 250)

In [5]:
features, target = feat_select(train_data, test_data)
sm=SMOTEENN()
# train_data.replace(to_replace=-1,value=0,inplace=True)
train_data.replace(to_replace=np.nan, value=0, inplace=True)
train_data.replace(to_replace=-np.inf, value=0, inplace=True)
train_data.replace(to_replace=np.inf, value=0, inplace=True)

In [ ]:
X_resampled, y_resampled = sm.fit_resample(train_data[features], train_data[target])
X_resampled = pd.DataFrame(X_resampled, columns=features)
y_resampled = pd.DataFrame(y_resampled, columns=target)
X_resampled['is_trade'] = y_resampled['is_trade']
del y_resampled
gc.collect()

In [ ]:
best_iter = xgbCV(train_data, test_data)
print('最佳迭代次数：', best_iter)

In [ ]:
sub(train_data, test_data, best_iter)
gc.collect()

In [ ]:
bst = xgb.Booster({'nthread': 6})  # init model
bst.load_model('001.model')  # load data
X_test_set=xgb.DMatrix(test_data[features], missing=np.nan)
y_pred = bst.predict(X_test_set)

In [ ]:
y_true= test_data[target].values

In [ ]:
y_true.shape

In [ ]:
y_pred

In [ ]:
predictions = [round(value) for value in y_pred]
predictions=np.array(predictions).T
predictions.shape

In [ ]:
tp=0
tn=0
fp=0
fn=0
for i in range(0,119754):
    if predictions[i]==y_true.T[0][i]:
        if predictions[i]==1:
            tp+=1
        else:
            tn+=1
    else:
        if predictions[i]==1:
            fp+=1
        else:
            fn+=1
    
print(tp,tn,fp,fn)
        

In [ ]:
y_true.T[0]


In [ ]:
#输出模型的一些结果
print ("\n关于现在这个模型")
print ("准确率 : %.4f" % metrics.accuracy_score(y_true, predictions))
print("召回率:%.4f" % metrics.recall_score(y_true,predictions))
print("精度:%.4f" % metrics.precision_score(y_true, predictions))
print("F1:%.4f" % metrics.f1_score(y_true, predictions))
print ("AUC 得分 (训练集): %f" % 0.905728)
print ("AUC 得分 (测试集): %f" % 0.767387)

In [ ]:
xgb.plot_importance(gbm)

In [ ]:
importance = gbm.get_fscore()
for key,value in importance.items():
    print(key,value)